# Item-Based Collaborative Filtering

Shout out to https://github.com/zbeyza/collaborative_filtering/tree/main

**This method involves examining either the similarity of products or the ratings assigned by users to those products, and subsequently providing recommendations based on this analysis. An online streaming platform is looking to construct a recommendation system utilizing collaborative filtering. The goal is to generate recommendations that align with the preferences of the user community. Specifically, when users favor a movie, the system aims to suggest other movies with similar liking patterns, enhancing the overall user experience.**

In [1]:
import pandas as pd 

## Data Prepration

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [5]:
movies = pd.read_csv('../datasets/ml-20m/movies.csv')
ratings = pd.read_csv('../datasets/ml-20m/ratings.csv')
df = movies.merge(ratings, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09


## Creating User-Movie DataFrame

Number of ratings for movies

In [49]:
rating_counts = df.groupby('title').size().reset_index(name='count').sort_values(by='count', ascending=False)
rating_counts

,title,count
19154,Pulp Fiction (1994),67310
8905,Forrest Gump (1994),66172
21140,"Shawshank Redemption, The (1994)",63366
21360,"Silence of the Lambs, The (1991)",63299
12881,Jurassic Park (1993),59715
...,...,...
23790,The Italian Connection (1972),1
23788,The Iron Commissioner (1978),1
23787,The Invitation (2003),1
23786,The Invisible Frame (2009),1


Movies with less than 1000 ratings

In [41]:
rare_movies = rating_counts[rating_counts['count'] <= 1000].index

Movies with more than 1000 ratings

In [44]:
common_movies = df[~df['title'].isin(rare_movies)]
common_movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,1.427744e+09
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,1.427745e+09
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,1.427745e+09
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,1.427745e+09


Pivoting the DataFrame

In [46]:
users_movies = common_movies.pivot_table(index=['userId'], columns=['title'], values='rating')
users_movies

/var/folders/7q/lktxww195rs079d3gtxnyfmw0000gp/T/ipykernel_11422/356740312.py:1: PerformanceWarning: The following operation may generate 3701779397 cells in the resulting pandas object.
  users_movies = common_movies.pivot_table(index=['userId'], columns=['title'], values='rating')


title,"""Great Performances"" Cats (1998)",#chicagoGirl: The Social Network Takes on a Dictator (2013),$ (Dollars) (1971),$5 a Day (2008),$9.99 (2008),$ellebrity (Sellebrity) (2012),'71 (2014),'Hellboy': The Seeds of Creation (2004),"'Human' Factor, The (Human Factor, The) (1975)",'Neath the Arizona Skies (1934),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Twas the Night Before Christmas (1974),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),(A)sexual (2011),(Absolutions) Pipilotti's Mistakes ((Entlastungen) Pipilottis Fehler) (1988),(Untitled) (2009),*batteries not included (1987),+1 (2013),"...All the Marbles (California Dolls, The) (1981)",...And God Spoke (1993),...And Justice for All (1979),...tick... tick... tick... (1970),.45 (2006),009 Re: Cyborg (2012),1 (2013),"1, 2, 3, Sun (Un, deuz, trois, soleil) (1993)",1-900 (06) (1994),10 (1979),10 000 timmar (2014),10 Items or Less (2006),10 MPH (2007),10 Mountains 10 Years (2010),10 Questions for the Dalai Lama (2006),10 Rillington Place (1971),10 Things I Hate About You (1999),10 Years (2011),10 minutes (10 minuta) (2002),10 to Midnight (1983),"10,000 BC (2008)",10.000 Km (Long Distance) (2014),10.5 (2004),10.5: Apocalypse (2006),100 Bloody Acres (2012),100 Feet (2008),100 Girls (2000),100 Men and a Girl (One Hundred Men and a Girl) (1937),100 Rifles (1969),100 Ways to Murder Your Wife (Sha qi er ren zu) (1986),100 Years at the Movies (1994),100 Years of Evil (2010),"1000 Eyes of Dr. Mabuse, The (Die 1000 Augen des Dr. Mabuse) (1960)",1000 Journals (2007),1001 Nights (1990),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),102 Minutes That Changed America (2008),1066 (2009),10th & Wolf (2006),"10th Judicial Court: Judicial Hearings, The (10e chambre - Instants d'audience) (2004)","10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)",11 Flowers (Wo 11) (2011),11 Harrowhouse (1974),11 x 14 (1977),"11'09""01 - September 11 (2002)",11-11-11 (11-11-11: The Prophecy) (2011),11.6 (2013),112 Weddings (2013),11:14 (2003),"11th Hour, The (2007)",12 (2007),12 Angry Men (1957),12 Angry Men (1997),12 Dates of Christmas (2011),12 Days of Terror (2005),"12 Dogs of Christmas, The (2005)",12 O'Clock Boys (2013),12 Rounds (2009),12 Rounds 2: Reloaded (2013),12 Storeys (Shier lou) (1997),12 Years a Slave (2013),120 (2008),127 Hours (2010),12:01 (1993),12:01 PM (1990),12:08 East of Bucharest (A fost sau n-a fost?) (2006),12th & Delaware (2010),13 (2010),13 Assassins (Jûsan-nin no shikaku) (2010),13 Beloved (13 game sayawng) (2006),13 Fighting Men (1960),"13 Frightened Girls! (Candy Web, The) (1963)",13 Ghosts (1960),13 Going on 30 (2004),13 Rue Madeleine (1947),13 Sins (2014),13 Tzameti (2005),13/13/13 (2013),13B (2009),13Hrs (2010),"13th Letter, The (1951)","13th Warrior, The (1999)",14 Blades (Jin yi wei) (2010),14 Hours (Fourteen Hours) (1951),1408 (2007),1492: Conquest of Paradise (1992),15 (2005),15 Minutes (2001),16 Acres (2012),16 Blocks (2006),16 Years of Alcohol (2003),16 to Life (2009),1612: Khroniki smutnogo vremeni (2007),17 Again (2009),17 Girls (17 filles) (2011),1776 (1972),18 Again! (1988),18 Fingers of Death! (2006),18 Years Later (Diciotto anni dopo) (2010),18 Years Old and Rising (J'aime regarder les filles) (2011),180° South (180 Degrees South) (180° South: Conquerors of the Useless) (2010),187 (One Eight Seven) (1997),1900 (Novecento) (1976),1911 (Xinhai geming) (2011),1939: Hollywood's Greatest Year (2009),1941 (1979),1969 (1988),1971 (2014),1981 (2009),1984 (1956),1984 (Nineteen Eighty-Four) (1984),1987 (2014),1990: The Bronx Warriors (1990: I guerrieri del Bronx) (1982),1991: The Year Punk Broke (1992),"19th Wife, The (2010)",1½ Knights - In Search of the Ravishing Princess Herzelinde (2008),2 Become 1 (Tin sun yut dui) (2006),2 Days in New York (2012),2 Da

## Item-Based Movie Recommendation

The users-movies matrix has been created (with user IDs as rows and movie titles as columns). Now we can find similar movies, simply by looking at the correlations between the movies and user ratings. Let's look at one example:

In [54]:
rating_counts[rating_counts['title'].str.contains("Good Will Hunting")]

,title,count
9940,Good Will Hunting (1997),28324


In [63]:
users_movies.corrwith(users_movies["Good Will Hunting (1997)"]).sort_values(ascending=False).head(10)

# Alternatively, to randomly select a movie:
# users_movies.corrwith(users_movies[pd.Series(users_movies.columns).sample(1).values[0]]).sort_values(ascending=False).head(10)

title
Disorder (2009)                                              1.0
Moon in the Gutter, The (La lune dans le caniveau) (1983)    1.0
Entre ses mains (2005)                                       1.0
Monsieur Batignole (2002)                                    1.0
Money as Debt (2006)                                         1.0
Monday Morning (Lundi matin) (2002)                          1.0
Every Girl Should Be Married (1948)                          1.0
Triangle (Tie saam gok) (2007)                               1.0
Miracle at Oxford (True Blue) (1996)                         1.0
Evidence of Blood (1998)                                     1.0
dtype: float64